In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t

data = pd.read_csv('17_sem_nal_Izak_Jenko/Kibergrad.csv')
pd.options.display.max_rows = 15

In [2]:
data

TIP  CLANOV  OTROK  DOHODEK  CETRT  IZOBRAZBA
0        1       2      0    43450      1         39
1        1       2      0    79000      1         40
2        1       2      0    51306      1         39
3        1       4      2    24850      1         41
4        1       4      2    65145      1         43
...    ...     ...    ...      ...    ...        ...
43881    1       5      3    30800      4         36
43882    1       2      0    16684      4         34
43883    1       2      0    12000      4         39
43884    1       2      0    50022      4         34
43885    1       9      7    21082      4         39

[43886 rows x 6 columns]

In [3]:
# a) 

# velikost populacije
N = data['DOHODEK'].count()

# populacijsko povprečje
mu = data['DOHODEK'].mean()

# populacijska varianca
sigma2 = sum((data['DOHODEK'] - mu)**2)/N

# standardna napaka
n = 400
se = np.sqrt(sigma2 * (1 - (n - 1)/(N - 1))/n)

# enostavni slučajni vzorec velikosti n = 400 (brez vračanja)
# torej enote vzorca niso neodvisne so pa enako porazdeljene
# vzorec = data.sample(n = n)

vzorec = pd.read_csv('vzorec.csv')

# vzorčno povprečje - nepristranska cenilka za mu
X_bar = vzorec['DOHODEK'].mean()

# cenlika za standardno napako (ocenimo jo na podlagi vzorca)
s2 = sum((vzorec['DOHODEK'] - X_bar)**2)/(n - 1)
se_hat = np.sqrt(s2 * (1 - n/N)/n)


**Vprašanje:** Ali privzamem, da je standardna napaka znana, ali uporabim oceno od zgoraj? 
$$ a = b$$

In [10]:
# X_bar je vsota n enako porazdeljenih (ampak odvisnih) s.s., zato je po CLI
# (X_bar - mu)/se porazdeljen približno standardno normalno N(0,1) oz. X_bar je porazdeljen
# priblizno N(mu, se^2).

alpha = 0.05

# interval zaupanja za populacijsko povprečje (mu) pod predpostavko, da poznamo 
# populacijsko varianco sigma2, torej tocno standardno napako se. 
I = (X_bar - se * norm.ppf(1 - alpha/2), X_bar + se * norm.ppf(1 - alpha/2))
# Verjetnost, da je mu \in I je 95% pri stopnji tveganja 5%

#dolžina intervala
print(I[1] - I[0])

# (aproksimativni) interval zaupanja za populacijsko povprečje (mu), kadar ne poznamo populacijske 
# variance in namesto tega uporabimo oceno za standardno napako.
I_aproks = (X_bar - se_hat * norm.ppf(1 - alpha/2), X_bar + se_hat * norm.ppf(1 - alpha/2))

# dolžina aproksimativnega intervala
print(I_aproks[1] - I_aproks[0])

# studentov interval zaupanja
I_student = (X_bar - se_hat * t.ppf(1 - alpha/2, n - 1), X_bar + se_hat * t.ppf(1 - alpha/2, n - 1))
# dolžina je
I_student[1] - I_student[0]
 

6250.57629231918
6716.139358708082


6736.573627188758

$$
(\bar X - se \Phi^{-1}(1 - \alpha/2), \bar X + se \Phi^{-1}(1 - \alpha/2))
$$

In [5]:
# Test da vidimo da vse deluje
# Če nastavimo (aproksimativni) inteval zaupanja 
#   (X_bar - se_hat * phi^-1(1 - alpha/2), X_bar + se_hat * phi^-1(1 - alpha/2)),
# pričakujemo, da bo v 95% ta interval pokril populacijsko povprečje mu

counter = 0
for i in range(100):
    vzorec = data.sample(n = n)
    X_bar = vzorec['DOHODEK'].mean()
    s2 = sum((vzorec['DOHODEK'] - X_bar)**2)/(n - 1)
    se_hat = np.sqrt(s2 * (1 - n/N)/n)

    I = (X_bar - se_hat * norm.ppf(1 - alpha/2), X_bar + se_hat * norm.ppf(1 - alpha/2))
    if I[0] < mu and mu < I[1]:
        counter += 1
print(counter)

# vidimo, da se to res zgodi v priblizno 95/100 primerih...


96


In [6]:
# b) Stratumi

# najprej je treba izračunati proporcionalno alokacijo

# velikost stratumov
N_strat = [data[data['CETRT'] == i]['DOHODEK'].count() for i in range(1,5)]
# n_strat = [vzorec[vzorec['CETRT'] == i]['DOHODEK'].count() for i in range(1,5)]
W = [N_i/N for N_i in N_strat]

# populacijska povprečja po stratumih
mu_strat = [data[data['CETRT'] == i]['DOHODEK'].mean() for i in range(1,5)]

# populacijske variance po stratumih
sigma2_strat = [sum((data[data['CETRT'] == i]['DOHODEK'] - mu_strat[i-1])**2)/N_strat[i-1] for i in range(1,5)]

# proporcionalana alokacija
print([W[i]*n for i in range(4)]) # dve največji vrednosti zaokrožimo navzgor, ostali dve pa navzdol
n_strat = [92, 95, 123, 90]

# sedaj glede na to proporcionalno alokacijo izberemo vzorec
vzorec_strat = [
    data[data['CETRT'] == i].sample(n=n_strat[i-1]) for i in range(1,5)
]

# stratumska povprečja vzorca
X_bar_strat = [sum(vzorec_strat[i]['DOHODEK'])/n_strat[i] for i in range(4)]
X_bar = sum(W[i]*X_bar_strat[i] for i in range(4))

[92.50330401494782, 94.69990429749807, 122.65414938704826, 90.14264230050586]
